> Proszę zaimplementować funkcję wstawiającą dowolny element do kopca binarnego.

Ponieważ nie jest powiedziane, o jaki kopiec chodzi (Min Heap, czy Max Heap), poniżej prezentuję rozwiązanie dla obu kopców. W rozwiązaniach znajduje się zarówno wersja obiektowa oraz funkcyjna. W poniższych implementacjach ograniczam się do koniecznych metod do utworzenia kopca oraz dodania wartości (nie załączam metod, które usuwają odpowiednio największą/najmniejszą wartość lub ją odczytują). Pełna implementacja kopców znajduje się w <a href="../../../%23Kolokwium/Jupyter-notebooks/Drzewiaste struktury danych.ipynb">TYM</a> pliku.

##### Funkcja wypisująca kopiec binarny (nie jest częścią algorytmu)

In [1]:
def complete_tree_string(values):
    if values:
        just = 0
        data = []

        limit = 1
        values_row = []
        branches_row = []
        prev_nodes = 0

        for i in range(1, len(values) + 1):
            curr_nodes = i - prev_nodes
            val_str = str(values[i-1])
            just = max(just, len(val_str))
            values_row.append(val_str)
            right_child_idx = 2 * i
            left_child_idx = right_child_idx - 1
            if left_child_idx < len(values):
                branches_row.append('/')
            if right_child_idx < len(values):
                branches_row.append('\\')

            if curr_nodes == limit: 
                prev_nodes = i
                limit *= 2
                data.append([values_row, branches_row])
                values_row = []
                branches_row = []

        if values_row:
            data.append([values_row, branches_row])

        begin_sep = sep = 3 if just % 2 else 2
        data_iter = iter(data[::-1])
        result = [''] * (len(data) * 2 - 1)
        result[-1] = (' ' * sep).join(val.center(just) for val in next(data_iter)[0])

        # Format the tree string
        for i, (values, branches) in enumerate(data_iter):
            mul = 2 * i + 1
            # Values
            indent = (2 ** (i + 1) - 1) * (just + begin_sep) // 2
            sep = 2 * sep + just
            result[-(mul + 2)] = f"{' ' * indent}{(' ' * sep).join(val.center(just) for val in values)}"
            # Branches
            branch_indent = (3 * indent + just) // 4
            branches_row = []
            d_indent = indent - branch_indent
            branches_sep = ' ' * (2 * (d_indent - 1) + just)
            for i in range(0, len(branches), 2):
                branches_row.append(f"{branches[i]}{branches_sep}{branches[i + 1] if i + 1 < len(branches) else ''}")
            result[-(mul + 1)] = f"{' ' * branch_indent}{(' ' * (sep - 2 * d_indent)).join(branches_row)}"

        return '\n'.join(result)
    else:
        return ''

## Max Heap

### Implementacja algorytmu #1 (obiektowy kopiec)

In [2]:
class MaxHeap:
    def __init__(self, values=None):
        if values:
            self.heap = list(values) # We make a copy of values in order not to modify them
            self.build_heap()
        else:
            self.heap = []
        
    def __str__(self):  # A 'complete_tree_string' function is required in order to ensure that printing works
        return complete_tree_string(self.heap)
    
    def __bool__(self):
        return bool(self.heap)
    
    @property
    def heap_size(self):
        return len(self.heap)
    
    @staticmethod
    def parent_idx(curr_idx):
        return (curr_idx - 1) // 2
    
    @staticmethod
    def left_child_idx(curr_idx):
        return curr_idx * 2 + 1
    
    @staticmethod
    def right_child_idx(curr_idx):
        return curr_idx * 2 + 2
    
    def insert(self, val: object):
        # Add a value as the last node of out Complete Binary Tree
        self.heap.append(val)
        # Fix heap in order to satisfy a max-heap property
        self._heapify_up(self.heap_size - 1)
    
    def swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]
        
    def _heapify_up(self, curr_idx, end_idx=0):
        while curr_idx > end_idx:
            parent_idx = self.parent_idx(curr_idx)
            if self.heap[curr_idx] > self.heap[parent_idx]:
                self.swap(curr_idx, parent_idx)
            curr_idx = parent_idx
        
    def _heapify_down(self, curr_idx, end_idx):
        # Loop till the current node has a child larger than itself
        # We assume that when we enter a node which both children are
        # smaller than this node, a subtree which a current node is a
        # root of must fulfill a max-heap property
        while True:
            l = self.left_child_idx(curr_idx)
            r = self.right_child_idx(curr_idx)
            largest_idx = curr_idx

            if l < end_idx:
                if self.heap[l] > self.heap[curr_idx]: 
                    largest_idx = l
                if r < end_idx and self.heap[r] > self.heap[largest_idx]:
                    largest_idx = r

            if largest_idx != curr_idx:
                self.swap(curr_idx, largest_idx)
                curr_idx = largest_idx
            else:
                break
        
    def build_heap(self):
        for i in range(self.heap_size // 2 - 1, -1, -1):
            self._heapify_down(i, self.heap_size)

###### Kilka testów

In [3]:
mh = MaxHeap(range(3))
print(mh, end='\n\n')
mh.insert(2)
print(mh, end='\n\n')
mh.insert(0)
print(mh, end='\n\n')
mh.insert(6)  # See how nodes were swapped after inserting this value
print(mh, end='\n\n')
mh.insert(7)
print(mh, end='\n\n')
for i in range(8, 200, 30):
    mh.insert(i)
print(mh, end='\n\n')

  2
 / \
1   0

      2
    /   \
  2       0
 / 
1

      2
    /   \
  2       0
 / \
1   0

      6
    /   \
  2       2
 / \     / 
1   0   0

      7
    /   \
  2       6
 / \     / \
1   0   0   2

                     188
                /           \
          68                     158
       /     \                 /     \
    7           38          98         128
   / \         / \         / \         / 
 1     2     0     8     0     6     2 



## Implementacja algorytmu #2 (funkcyjny kopiec)

In [4]:
_left = lambda i: 2 * i + 1
_right = lambda i: 2 * i + 2
_parent = lambda i: (i - 1) // 2


# Swap values in an array in order to satisfy a max-heap property
def build_max_heap(values: list):
    for i in range(len(values) // 2 - 1, -1, -1):
        _heapify_down(values, i, len(values))
        
        
def insert_to_max_heap(heap: list, val: object):
    # Add a value as the last node of out Complete Binary Tree
    heap.append(val)
    # Fix heap in order to satisfy a max-heap property
    _heapify_up(heap, len(heap) - 1)
    

def print_max_heap(heap: list, *args, **kwargs):
    print(complete_tree_string(heap), *args, **kwargs)
    
    
def _swap(heap: list, i, j):
    heap[i], heap[j] = heap[j], heap[i]
    
    
def _heapify_up(heap: list, curr_idx: 'heapify begin index', end_idx: 'heapify end index' = 0):
    while curr_idx > end_idx:
        parent_idx = _parent(curr_idx)
        if heap[curr_idx] > heap[parent_idx]:
            _swap(heap, curr_idx, parent_idx)
        curr_idx = parent_idx
            
    
def _heapify_down(heap: list, curr_idx: 'heapify begin index', end_idx: 'heapify end index'):
    # Loop till the current node has a child larger than itself
    # We assume that when we enter a node which both children are
    # smaller than this node, a subtree which a current node is a
    # root of must fulfill a max-heap property.
    while True:
        i = _left(curr_idx)
        j = _right(curr_idx)
        k = curr_idx
        
        if i < end_idx:
            if heap[i] > heap[k]:
                k = i
            if j < end_idx and heap[j] > heap[k]:
                k = j
                
        if k == curr_idx: return
        # Swap the current with the largest child
        heap[curr_idx], heap[k] = heap[k], heap[curr_idx]
        curr_idx = k

###### Kilka testów

In [5]:
mh = list(range(3))
build_max_heap(mh)
print_max_heap(mh, end='\n\n')
insert_to_max_heap(mh, 2)
print_max_heap(mh, end='\n\n')
insert_to_max_heap(mh, 0)
print_max_heap(mh, end='\n\n')
insert_to_max_heap(mh, 6)
print_max_heap(mh, end='\n\n')
insert_to_max_heap(mh, 7)
print_max_heap(mh, end='\n\n')
for i in range(8, 200, 30):
    insert_to_max_heap(mh, i)
print_max_heap(mh, end='\n\n')

  2
 / \
1   0

      2
    /   \
  2       0
 / 
1

      2
    /   \
  2       0
 / \
1   0

      6
    /   \
  2       2
 / \     / 
1   0   0

      7
    /   \
  2       6
 / \     / \
1   0   0   2

                     188
                /           \
          68                     158
       /     \                 /     \
    7           38          98         128
   / \         / \         / \         / 
 1     2     0     8     0     6     2 



## Min Heap

### Implementacja algorytmu #1 (obiektowy kopiec)

In [6]:
class MinHeap:
    def __init__(self, values=None):
        if values:
            self.heap = list(values) # We make a copy of values in order not to modify them
            self.build_heap()
        else:
            self.heap = []
        
    def __str__(self):  # A 'complete_tree_string' function is required in order to ensure that printing works
        return complete_tree_string(self.heap)
    
    def __bool__(self):
        return bool(self.heap)
    
    @property
    def heap_size(self):
        return len(self.heap)
    
    @staticmethod
    def parent_idx(curr_idx):
        return (curr_idx - 1) // 2
    
    @staticmethod
    def left_child_idx(curr_idx):
        return curr_idx * 2 + 1
    
    @staticmethod
    def right_child_idx(curr_idx):
        return curr_idx * 2 + 2
    
    def insert(self, val: object):
        # Add a value as the last node of out Complete Binary Tree
        self.heap.append(val)
        # Fix heap in order to satisfy a max-heap property
        self._heapify_up(self.heap_size - 1)
    
    def swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]
        
    def _heapify_up(self, curr_idx, end_idx=0):
        while curr_idx > end_idx:
            parent_idx = self.parent_idx(curr_idx)
            if self.heap[curr_idx] < self.heap[parent_idx]:
                self.swap(curr_idx, parent_idx)
            curr_idx = parent_idx
        
    def _heapify_down(self, curr_idx, end_idx):
        # Loop till the current node has a child larger than itself
        # We assume that when we enter a node which both children are
        # greater than this node, a subtree which a current node is a
        # root of must fulfill a min-heap property
        while True:
            l = self.left_child_idx(curr_idx)
            r = self.right_child_idx(curr_idx)
            largest_idx = curr_idx

            if l < end_idx:
                if self.heap[l] < self.heap[curr_idx]: 
                    largest_idx = l
                if r < end_idx and self.heap[r] < self.heap[largest_idx]:
                    largest_idx = r

            if largest_idx != curr_idx:
                self.swap(curr_idx, largest_idx)
                curr_idx = largest_idx
            else:
                break
        
    def build_heap(self):
        for i in range(self.heap_size // 2 - 1, -1, -1):
            self._heapify_down(i, self.heap_size)

###### Kilka testów

In [7]:
mh = MinHeap(range(3))
print(mh, end='\n\n')
mh.insert(2)
print(mh, end='\n\n')
mh.insert(0)
print(mh, end='\n\n')
mh.insert(6)  # See how nodes were swapped after inserting this value
print(mh, end='\n\n')
mh.insert(7)
print(mh, end='\n\n')
for i in range(8, 200, 30):
    mh.insert(i)
print(mh, end='\n\n')

  0
 / \
1   2

      0
    /   \
  1       2
 / 
2

      0
    /   \
  0       2
 / \
2   1

      0
    /   \
  0       2
 / \     / 
2   1   6

      0
    /   \
  0       2
 / \     / \
2   1   6   7

                      0 
                /           \
          0                       2 
       /     \                 /     \
    2           1           6           7 
   / \         / \         / \         / 
 8     38    68    98   128   158   188



## Implementacja algorytmu #2 (funkcyjny kopiec)

In [8]:
_left = lambda i: 2 * i + 1
_right = lambda i: 2 * i + 2
_parent = lambda i: (i - 1) // 2


# Swap values in an array in order to satisfy a max-heap property
def build_min_heap(values: list):
    for i in range(len(values) // 2 - 1, -1, -1):
        _heapify_down(values, i, len(values))
        
        
def insert_to_min_heap(heap: list, val: object):
    # Add a value as the last node of out Complete Binary Tree
    heap.append(val)
    # Fix heap in order to satisfy a max-heap property
    _heapify_up(heap, len(heap) - 1)
    

def print_min_heap(heap: list, *args, **kwargs):
    print(complete_tree_string(heap), *args, **kwargs)
    
    
def _swap(heap: list, i, j):
    heap[i], heap[j] = heap[j], heap[i]
    
    
def _heapify_up(heap: list, curr_idx: 'heapify begin index', end_idx: 'heapify end index' = 0):
    while curr_idx > end_idx:
        parent_idx = _parent(curr_idx)
        if heap[curr_idx] < heap[parent_idx]:
            _swap(heap, curr_idx, parent_idx)
        curr_idx = parent_idx
            
    
def _heapify_down(heap: list, curr_idx: 'heapify begin index', end_idx: 'heapify end index'):
    # Loop till the current node has a child larger than itself
    # We assume that when we enter a node which both children are
    # greater than this node, a subtree which a current node is a
    # root of must fulfill a min-heap property.
    while True:
        i = _left(curr_idx)
        j = _right(curr_idx)
        k = curr_idx
        
        if i < end_idx:
            if heap[i] < heap[k]:
                k = i
            if j < end_idx and heap[j] < heap[k]:
                k = j
                
        if k == curr_idx: return
        # Swap the current with the largest child
        heap[curr_idx], heap[k] = heap[k], heap[curr_idx]
        curr_idx = k

###### Kilka testów

In [9]:
mh = list(range(3))
build_min_heap(mh)
print_min_heap(mh, end='\n\n')
insert_to_min_heap(mh, 2)
print_min_heap(mh, end='\n\n')
insert_to_min_heap(mh, 0)
print_min_heap(mh, end='\n\n')
insert_to_min_heap(mh, 6)
print_min_heap(mh, end='\n\n')
insert_to_min_heap(mh, 7)
print_min_heap(mh, end='\n\n')
for i in range(8, 200, 30):
    insert_to_min_heap(mh, i)
print_min_heap(mh, end='\n\n')

  0
 / \
1   2

      0
    /   \
  1       2
 / 
2

      0
    /   \
  0       2
 / \
2   1

      0
    /   \
  0       2
 / \     / 
2   1   6

      0
    /   \
  0       2
 / \     / \
2   1   6   7

                      0 
                /           \
          0                       2 
       /     \                 /     \
    2           1           6           7 
   / \         / \         / \         / 
 8     38    68    98   128   158   188

